


---

**Installation des librairies nécessaires aux calculs**


---





In [106]:
import sys
import subprocess
import pkg_resources

required = {'numpy', 'pandas', 'numba', 'matplotlib', 'plotly', 'seaborn', 'yfinance'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

import yfinance as yf
import numpy as np
import pandas as pd
import numba as nb
import plotly.graph_objects as go

---

**Liaison à l'api de Yahoo Finance pour récupérer les données des entreprises**

Ceci nous permettera d'avoir des données à jour et plus de fléxibilité sur les actifs de notre portefeuille ainsi que les historiques des prix.
---



In [43]:
tickers = "CS.PA BNP.PA CO.PA ENGI.PA MC.PA AC.PA ORA.PA PEUG.PA RNO.PA TTE.PA VIE.PA VIV.PA"
data = yf.download(tickers, start="2018-01-02", end="2019-12-23").dropna()
data

[*********************100%***********************]  12 of 12 completed


Adj Close                                                          \
                AC.PA     BNP.PA      CO.PA      CS.PA    ENGI.PA       MC.PA   
Date                                                                            
2018-01-02  41.274788  43.358849  45.164017  17.122972   9.910755  220.724670   
2018-01-03  41.113411  43.742920  45.711998  17.102226   9.952542  222.051025   
2018-01-04  41.388702  44.532028  46.242298  17.454918  10.109248  224.658005   
2018-01-05  41.550079  44.615826  47.108456  17.638182  10.164968  228.225464   
2018-01-08  42.129143  45.041798  45.888763  17.852566  10.231131  224.566513   
...               ...        ...        ...        ...        ...         ...   
2019-12-16  41.630001  42.091171  43.830002  19.314238  11.063114  388.774719   
2019-12-17  42.250000  41.996235  43.230000  19.391619  11.127324  382.084778   
2019-12-18  41.400002  42.146557  42.070000  19.414833  10.942245  380.376709   
2019-12-19  41.349998  42.336441  42.439999  19.376143  10.953578  381.705200   
2019-12-20  41.139999  42.375999  42.090000  19.658585  11.142432  389.533783   

                                                        ...  Volume            \
               ORA.PA    PEUG.PA     RNO.PA     TTE.PA  ...   CO.PA     CS.PA   
Date                                                    ...                     
2018-01-02  11.057041  88.922417  70.844910  33.381302  ...  238259   5197529   
2018-01-03  11.034195  87.593895  71.090073  33.743668  ...  362125   4878529   
2018-01-04  11.038003  89.276688  72.045380  34.410431  ...  309048   6590382   
2018-01-05  11.133192  89.985237  73.719276  34.660465  ...  394755   4387081   
2018-01-08  11.194112  90.870911  74.792946  34.812653  ...  610009   6061618   
...               ...        ...        ...        ...  ...     ...       ...   
2019-12-16  11.187417  97.098648  40.629208  38.260548  ...  432234   4203177   
2019-12-17  11.271250  96.914055  41.153061  38.660736  ...  226494   6605781   
2019-12-18  11.128734  98.575432  41.082272  38.522060  ...  379711   5155617   
2019-12-19  11.111967  97.098648  40.992603  38.831127  ...  269533   5966440   
2019-12-20  11.153883  96.729454  41.478699  38.886597  ...  404790  12656176   

                                                                             \
             ENGI.PA    MC.PA    ORA.PA PEUG.PA   RNO.PA    TTE.PA   VIE.PA   
Date                                                                          
2018-01-02   4784430   631294   5623409    2907   872516   4997867  1300641   
2018-01-03   5666606   521585   5713414    3132   773646   6782657  2131351   
2018-01-04   5665448   611052   7180972    4835  1149993   8356524  1906862   
2018-01-05   5086093   454725   5739730    7549   926891   5799220  1817944   
2018-01-08   4068532   536733   4569150    6314  1022507   3827349  1505979   
...              ...      ...       ...     ...      ...       ...      ...   
2019-12-16   6519847   511002   7407998    5869  1125306   4413533  1639695   
2019-12-17   6936432   734684   8097112    5782  1405984   5599136  2213959   
2019-12-18   6483653   593446  10206343    7622  1186823   5332920  1500795   
2019-12-19   4975010   429682   8574242    3125   960099   4838642  1534954   
2019-12-20  11715448  1290604  14658568    3443  2551581  11608763  2235832   

                     
             VIV.PA  
Date                 
2018-01-02  3375024  
2018-01-03  4585272  
2018-01-04  4282574  
2018-01-05  3985639  
2018-01-08  3817102  
...             ...  
2019-12-16  2512022  
2019-12-17  2831898  
2019-12-18  2922676  
2019-12-19  1987347  
2019-12-20  6647521  

[505 rows x 72 columns]

Pour chaque actif, nous disposons des données suivantes : (Adj close, Close, Open, High, Low, Volume)

In [44]:
data.columns

MultiIndex([('Adj Close',   'AC.PA'),
            ('Adj Close',  'BNP.PA'),
            ('Adj Close',   'CO.PA'),
            ('Adj Close',   'CS.PA'),
            ('Adj Close', 'ENGI.PA'),
            ('Adj Close',   'MC.PA'),
            ('Adj Close',  'ORA.PA'),
            ('Adj Close', 'PEUG.PA'),
            ('Adj Close',  'RNO.PA'),
            ('Adj Close',  'TTE.PA'),
            ('Adj Close',  'VIE.PA'),
            ('Adj Close',  'VIV.PA'),
            (    'Close',   'AC.PA'),
            (    'Close',  'BNP.PA'),
            (    'Close',   'CO.PA'),
            (    'Close',   'CS.PA'),
            (    'Close', 'ENGI.PA'),
            (    'Close',   'MC.PA'),
            (    'Close',  'ORA.PA'),
            (    'Close', 'PEUG.PA'),
            (    'Close',  'RNO.PA'),
            (    'Close',  'TTE.PA'),
            (    'Close',  'VIE.PA'),
            (    'Close',  'VIV.PA'),
            (     'High',   'AC.PA'),
            (     'High',  'BNP.PA'),
            



---

Nous allons nous baser sur la moyenne des prix du jour tel que la moyenne est égale au **(high + low)/2**.


---



In [45]:
data = (data['High'] + data['Low'])/2
prices = data.transpose().to_numpy() #Extraction des prix sous forme numpy afin de pouvoir accélérer le calcul en utilisant la librarie numba (cette dernière n'est pas compatible avec pandas...)

In [46]:
data

,AC.PA,BNP.PA,CO.PA,CS.PA,ENGI.PA,MC.PA,ORA.PA,PEUG.PA,RNO.PA,TTE.PA,VIE.PA,VIV.PA
Date,,,,,,,,,,,,
2018-01-02,43.179998,62.180000,51.010000,24.637500,14.2750,243.400002,14.4600,100.149998,82.829998,46.125000,21.190001,22.375000
2018-01-03,43.485001,62.625000,51.539999,24.745000,14.2000,242.225006,14.4550,99.199997,84.320000,46.357500,21.224999,22.759999
2018-01-04,43.600000,63.500000,52.270000,25.087501,14.4350,245.300003,14.4825,99.700001,85.105000,47.187500,21.555000,23.145000
2018-01-05,43.834999,64.129997,52.960001,25.427500,14.5625,247.775002,14.5475,101.599998,86.380001,47.814999,21.785000,23.380000
2018-01-08,44.290001,64.424999,52.699999,25.782500,14.6575,246.175003,14.6575,102.500000,88.115002,48.062500,21.980000,23.680000
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-16,41.290001,52.814999,43.090000,24.897500,14.7100,408.900009,13.3300,104.500000,42.862499,48.125000,23.415000,25.349999
2019-12-17,42.105001,52.859999,43.480000,24.947499,14.6950,404.724991,13.4225,105.099998,43.189999,48.652500,23.675000,25.485000
2019-12-18,41.580000,53.100000,42.155001,25.065001,14.5950,401.574997,13.4000,106.199997,43.217501,48.842501,23.725000,25.490000




---
On calcule les ratios $\frac{S_{j+1} - S_j}{S_j}$

La fonction **pct_change()** de Pandas permet de faire cette opération fluidement !

---




In [5]:
import numpy as np
def calculate_ratios(data):
    return data.pct_change().dropna().transpose().to_numpy()
ratios = calculate_ratios(data)



---
On prépare la fonction qui calcul les $\mathrm{E}(R_i)$ 


---



In [6]:
def calculate_ER(Rs):
    return Rs.mean(axis = 1)
calculate_ER(ratios)

array([-4.07101541e-05, -2.09583326e-04, -1.58744652e-04,  9.82660631e-05,
        9.36164298e-05,  1.10093417e-03, -1.40378943e-04,  1.77799068e-04,
       -1.12187589e-03,  1.71917963e-04,  2.71060267e-04,  3.28754090e-04])

Définition de la fonction $\mathrm{VaR}_\alpha(X)$.

---



Cette version est optimisée, ci-dessous une comparaison du temps d'execution avec la fonctio numpy quantile.

On remarque qu'elle est **8x plus rapide sur une liste de 1000 elements**, et donne le même résultat !


---



In [78]:
@nb.njit()
def VaR(data_np, alpha):
    sorted_data = np.sort(data_np)
    index_var = int((1-alpha)*len(sorted_data))
    return -sorted_data[index_var]

In [93]:
sample = np.random.randint(1,2000,1000)

In [94]:
%timeit VaR(sample, 0.95)

21.2 µs ± 591 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [100]:
%timeit np.quantile(sample,0.05)

161 µs ± 32.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [96]:
VaR(sample, 0.95)

-89

In [99]:
-np.quantile(sample,0.05)

-89.0



---
Définition de la fonction produit scalaire qui est beaucoup plus rapide sur Numba que sur Numpy.


---



In [101]:
@nb.njit()
def mult(Rs, x):
    new_arr = np.zeros(Rs.shape[1])
    for j in range(Rs.shape[1]):
        for i in range(Rs.shape[0]):
            new_arr[j] += Rs[i][j] * x[i]
    return new_arr




---


La fonction **optimize** permet de minimiser le risque (VaR) sous condition de rendement annuel >= 5%.
le but de l'exercice est de résoudre le système suivant en x :

\begin{cases}
min_x(R_o(R_x)) \\
\mathrm{E}(R_x) > 5\%
\end{cases}

tq :
$P(R_o(R_x) + R_x < 0) > 95\%$

On définit la notion de lookback comme l'historique à prendre en compte afin d'estimer la VaR, il est intuitif de dire que le prix de l'action d'il y a 10 ans n'apporte pas d'information pertinente sur la situation actuel du marché. Donc il existe une durée maximal (lookback) à partir de laquel il n'est plus pertinent de prendre les données antérieure.
---



In [102]:
@nb.njit()
def optimize(ratios, alpha, lookback_period, objective_ER, n_iter):
    var_min = np.Inf
    for i in nb.prange(n_iter):
        x = np.random.uniform(0,1,len(ratios))
        x = x / np.sum(x)
        Rx = mult(ratios,x)
        ER_x = np.mean(Rx[-lookback_period:])
        VaR_x = VaR(Rx[-lookback_period:], alpha)
        if (1+ER_x)**(255) - 1 > objective_ER:
            if VaR_x < var_min:
                var_min = VaR_x
                opt_x = x
    return var_min, opt_x

Application pour le portefeuille du cours (12 actifs) avec $\alpha $ = 95% , $lookback$ = 255, un rendement minimal de 5% annuel et 1 million d'itération **(que 1 minute d'exécution grace à Numba et l'implémentation optimisée de la VaR !)**

In [103]:
optimize(ratios, 0.95,255, 0.05,1000000)

(0.00880966547278156,
 array([0.00313053, 0.00551467, 0.03595073, 0.1501957 , 0.20857514,
        0.03104577, 0.15682873, 0.04377939, 0.03096755, 0.01739936,
        0.16935691, 0.1472555 ]))



---


Nous retrouvons que **le minimum de risque** que nous **devons porter dans 95% des pires cas** afin de garantir un rendement de **5% annuel** est : **0.88% journalier.**


---





---

# **Backtesting avec rebalancement journalier**

---



Fonction helper qui permet de récupérer les derniers prix des actifs

In [12]:
@nb.njit()
def get_Si(S):
  returned_array = np.zeros(len(S))
  for i in range(len(returned_array)):
    returned_array[i] = S[i][-1]
  return returned_array

Fonction helper qui permet de récupérer l'historique des prix jusqu'à une date précise

In [13]:
@nb.njit()
def get_till_date(L,k):
  returned_array = np.zeros((L.shape[0],k))
  for i in range(L.shape[0]):
    for j in range(k):
      returned_array[i][j] = L[i][j]
  return returned_array




---

Nous changeons l'énoncer d'optimisation pour la partie backtesting, le but ici est de maximiser le gain tout en spécifiant nos limites de prise de risque sous forme de VaR.

Je suis peut être résilient aux mouvements du marché si ces derniers restent faible selon mon appréciation. Je me dis que je serais capable de tenir mes positions si la fluctuation journalière en négatif ne dépasse pas **max_VaR (ex : 3%)**, Donc si **un signal d'acha**t me permet d'avoir une position avec une **perte de 3% maximum** dans **5% des cas**, je suis preneur !

---



In [14]:
@nb.njit()
def optimize_ptf(ratios, alpha, lookback_period, max_VaR, n_iter):
    ER_max = -np.Inf
    for i in nb.prange(n_iter):
        x = np.random.uniform(0,1,len(ratios))
        x = x / np.sum(x)
        Rx = mult(ratios,x)
        ER_x = np.mean(Rx[-lookback_period:])
        VaR_x = VaR(Rx[-lookback_period:], alpha)
        if VaR_x< max_VaR:
            if ER_x > ER_max:
                ER_max = ER_x
                opt_x = x
    return ER_max, opt_x



---

Ci-dessous le moteur de backtesting, chaque jour j'éxecute un calcul d'optimisation qui me permet de trouver la répartition optimale de mes actifs tout en restant dans ma limite de risque.

---



In [118]:
def backtest_portfolio(init_val, ratios, prices, lookback_period, alpha, objective_ER):
  number_of_days = len(ratios[0])
  cash = init_val
  invested_V = 0.0
  total_V = invested_V + cash
  qte_opt = np.zeros(len(prices), dtype=np.float64)
  CASHs = np.zeros(number_of_days - (lookback_period+1), dtype=np.float64)
  Vs = np.zeros(number_of_days - (lookback_period+1), dtype=np.float64)
  for i in range(lookback_period+1, number_of_days):
    CASHs[i - (lookback_period+1)] = cash
    Vs[i - (lookback_period+1)] = total_V
    available_ratios = get_till_date(ratios, i)
    available_prices = get_till_date(prices, i)
    last_prices = get_Si(available_prices)
    total_V = cash + np.sum(qte_opt * last_prices)
    x_opt = optimize_ptf(available_ratios, alpha, lookback_period, 0.02, 10000)
    if np.isinf(x_opt[0]):
      v_opt = qte_opt * total_V
    else:
      v_opt = x_opt[1] * total_V
    qte_opt = (v_opt/last_prices).astype(np.int64)
    invested_V = np.sum(qte_opt * last_prices)
    cash = total_V - invested_V
  return CASHs, Vs







---


Vous pouvez jouer avec le moteur d'optimisation en choisissant les actifs que vous voulez en les espaçants dans la variable **ptf_tickers**.

Vous pouvez aussi choisir **start** et **end** date dans la **2e ligne**.

Et spécifier **un montant initial** de portefeuille à la **dernière ligne**.

Le backtest **prendra 30 secondes** pour **2 années** en moyenne.

Nous avons spécifié un lookback à 30 jours vu que nous rebalançant journalièrement.


---



In [134]:
ptf_tickers = "CS.PA BNP.PA CO.PA ENGI.PA MC.PA AC.PA ORA.PA PEUG.PA RNO.PA TTE.PA VIE.PA VIV.PA"
data_ptf = yf.download(ptf_tickers, start="2018-01-02", end="2019-12-23").dropna()
data_ptf = (data_ptf['High'] + data_ptf['Low'])/2
prices = data_ptf.transpose().to_numpy()
daily_R = calculate_ratios(data_ptf)
initial_value = 100000.0
lookback = 30
alpha = 0.95
max_daily_risk = 0.05

[*********************100%***********************]  12 of 12 completed


In [135]:
CSH, PTFV = backtest_portfolio(initial_value, daily_R, prices, lookback, alpha, max_daily_risk)



---


Il est interessent de comparer la performance de notre portefeuille à celle d'un indice, puisque il s'agît d'entreprises françaises, l'indice le plus pertinent serait le CAC40 avec une stratégie buy and hold.

Nous comparerons aussi avec l'indice américain S&P500.



---



In [157]:
CAC_40 = yf.download("^FCHI", start="2018-01-02", end="2019-12-23").dropna()
CAC_40 = CAC_40[-len(CSH):]
CAC_40

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-02-15,5211.709961,5255.080078,5204.439941,5222.520020,5222.520020,97295500
2018-02-16,5249.419922,5291.479980,5245.799805,5281.580078,5281.580078,101183700
2018-02-19,5295.330078,5297.370117,5249.890137,5256.180176,5256.180176,51447900
2018-02-20,5257.430176,5291.810059,5249.850098,5289.859863,5289.859863,75702000
2018-02-21,5278.479980,5303.810059,5255.770020,5302.169922,5302.169922,72945400
...,...,...,...,...,...,...
2019-12-16,5946.649902,6003.379883,5942.299805,5991.660156,5991.660156,77242800
2019-12-17,5989.479980,5989.479980,5955.359863,5968.259766,5968.259766,84477400
2019-12-18,5971.399902,5983.020020,5959.600098,5959.600098,5959.600098,80799300


In [156]:
SP500 = yf.download("^GSPC", start="2018-01-02", end="2019-12-23").dropna()
SP500 = SP500[-len(CSH):]
SP500

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-02-06,2614.780029,2701.040039,2593.070068,2695.139893,2695.139893,5917350000
2018-02-07,2690.949951,2727.669922,2681.330078,2681.659912,2681.659912,4646300000
2018-02-08,2685.010010,2685.270020,2580.560059,2581.000000,2581.000000,5333840000
2018-02-09,2601.780029,2638.669922,2532.689941,2619.550049,2619.550049,5690110000
2018-02-12,2636.750000,2672.610107,2622.449951,2656.000000,2656.000000,4069220000
...,...,...,...,...,...,...
2019-12-16,3183.629883,3197.709961,3183.629883,3191.449951,3191.449951,4070200000
2019-12-17,3195.399902,3198.219971,3191.030029,3192.520020,3192.520020,3842940000
2019-12-18,3195.209961,3198.479980,3191.139893,3191.139893,3191.139893,4029950000


In [158]:
SP500 = ((SP500['High'] + SP500['Low'])/2).to_numpy()
n_shares_SP = initial_value/SP500[0]
SP500_ptf = n_shares_SP*SP500

In [159]:
CAC_40 = ((CAC_40['High'] + CAC_40['Low'])/2).to_numpy()
n_shares_CAC = initial_value/CAC_40[0]
index_ptf = n_shares_CAC*CAC_40

In [160]:
fig = go.Figure(data=[go.Scatter(x=np.arange(0,len(PTFV),1), y=PTFV, name='Valeur totale du portefeuille',
                                 line=dict(color='#00BFFF'), marker=dict(color='#00BFFF')),
                      go.Scatter(x=np.arange(0,len(CSH),1), y=CSH, name='Cash restant',
                                 line=dict(color='#1d0ed1'), marker=dict(color='#1d0ed1')),
                      go.Scatter(x=np.arange(0,len(index_ptf),1), y=index_ptf, name='Indice CAC 40',
                                 line=dict(color='red'), marker=dict(color='red')),
                      go.Scatter(x=np.arange(0,len(SP500_ptf),1), y=SP500_ptf, name='Indice S&P500',
                                 line=dict(color='red'), marker=dict(color='green'))])

fig.update_layout(title='Valeur totale du portefeuille après rebalancement journalier',
                  xaxis_title='Temps (jours)',
                  yaxis_title='Valeur (Euro)',
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))
fig.update_layout(font=dict(family='Courier New, monospace', size=14))

fig.update_layout(xaxis_title_font_color='#636363',
                  yaxis_title_font_color='#636363',
                  xaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  yaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  title_x = 0.5)
fig.show()



---


Nous remarquons que le portefeuille s'est bien comporté malgrès que la période choisi est une période plutôt constante, mais entre **le 170e et 224e jours**, les marchés ont subît **une baisse** qui n'a pas été amplement reflété sur notre portefeulle **-18% pour le S&P500** et **-10% pour le CAC40** or notre portefeuille **n'a baissé que de 6.8% !**

---

